In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as plt
import numpy as np
from scipy.interpolate import interp1d

In [4]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def get_pt_onclick(event):
    global ix,iy, rgba
    ix, iy = event.xdata,event.ydata
    rgba = event.inaxes.get_images()[0].get_cursor_data(event).data
    print(ix, iy)
    
def get_pts_manual(event):
    fig.canvas.draw_idle()
    global ix,iy, pts_manual
    if event.key == 'm':
        plt.plot(ix,iy, 'x', color = 'tab:red') 
        pts_manual.append([ix,iy])


def cal_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, calpts
    if len(calpts)<4:
        #print(event.key)
        if event.key == 'enter':
            calpts.append([ix,iy])
            plt.plot(ix,iy, 'rx')
#             if len(calpts)>=4:
#                 plt.close(plt.gcf())


def bound_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, boundpts
    if len(boundpts)<4:
        #print(event.key)
        if event.key == 'b':
            boundpts.append([ix,iy])
            plt.plot(ix,iy, 'gx')

                
def line_color(event):
    global ix,iy,rgba,line_col
    if event.key == '-':
        print(event.key)
        line_col = rgba 
        
def mouse_eraser(event):
    fig.canvas.draw_idle()
    global ix,iy, pts
    if event.key == ' ':
        plt.plot(ix,iy, 'bo') 
        try:
            idx = np.where(((pts[0,:] - ix)**2 + (pts[1,:] - iy)**2)<300)[0]
            pts[:, idx] = 0
        except:
            print('bleh')

In [ ]:
filename = 'IMG_1830.png'

In [ ]:
#set calpts (enter), boundpts (b), line_col (-)

In [ ]:
img=mpimg.imread(filename)
fig = plt.figure(figsize=(10,5))
cid = fig.canvas.mpl_connect('button_press_event', get_pt_onclick)
kid = fig.canvas.mpl_connect('key_press_event', cal_points)
bid = fig.canvas.mpl_connect('key_press_event', bound_points)
lid = fig.canvas.mpl_connect('key_press_event', line_color)
eid = fig.canvas.mpl_connect('key_press_event', mouse_eraser)
mid = fig.canvas.mpl_connect('key_press_event', get_pts_manual)

ix,iy = 0,0
line_col = [0,0,0,0]
calpts = []
boundpts = []
#pts = np.copy(mask2)
pts_manual = []
imgplot = plt.imshow(img)
plt.gca().plot([300,300], 'ro')
plt.show()

In [ ]:
save_name = '5ps_18mW_250kHz'

In [ ]:
mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)
mask2 = np.copy(mask)
mask2[mask2>0.02] = 0
X,Y=np.meshgrid(np.arange(mask2.shape[1]),np.arange(mask2.shape[0]))
bb = np.array(boundpts)
x0 = np.min(bb[:, 0]) #x
y0 = np.max(bb[:, 1]) #y
x1 = np.max(bb[:, 0]) #x
y1 = np.min(bb[:, 1]) #y
roi = np.zeros(mask2.shape)
roi[np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[0],np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[1]] = 1
plt.figure(1)
pts = np.array([np.where(mask2*roi>0)[1],np.where(mask2*roi>0)[0]])
plt.plot(pts[0,:], pts[1,:], 'rx')
plt.show()

In [ ]:
#manual correction, erase (space), add (m)
pts = np.hstack((pts,np.array(pts_manual).T))
pts = np.vstack((pts[0,:][(pts[0,:]>x0)&(pts[1,:]<y0)],pts[1,:][(pts[0,:]>x0)&(pts[1,:]<y0)]))
plt.figure(1)
plt.plot(pts[0,:], pts[1,:], 'gx')
plt.show()

In [ ]:
#interpolate and save
calval = [-34,-24,0.01,0.1]
fx = interp1d([calpts[0][0],calpts[1][0]], [calval[0],calval[1]],bounds_error=None, fill_value = 'extrapolate')
fy = interp1d([calpts[2][1],calpts[3][1]],[calval[2],calval[3]],bounds_error=None, fill_value = 'extrapolate')
data = {}
data['x'] = fx(pts[0,:])
data['y'] = fy(pts[1,:])
np.save(save_name+'.npy',data)